In [ ]:
import transformers
from transformers import AutoModel, AutoImageProcessor
from transformers import pipeline
from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation, BaseImageProcessor
import numpy
import pathlib as pl
import PIL 
import datasets
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
image_processor = SegformerImageProcessor.from_json_file("./sip/preprocessor_config.json")
model = SegformerForSemanticSegmentation.from_pretrained("glacierscopessegmentation/glacier_segmentation_transformer")
data_path = pl.Path("./inference_dataset/inference_dataset.hf/")
ds = datasets.load_from_disk(data_path)
segmenter = pipeline(task="image-segmentation",image_processor=image_processor,model = model,device = "cuda:0")

In [ ]:
for i in ds:
    segments = segmenter(i["image"])
    red = np.array(segments[0]["mask"].convert("L"))
    green = np.array(segments[2]["mask"].convert("L"))
    blue = np.array(segments[1]["mask"].convert("L"))
    combo = np.stack([red, green, blue]).transpose(1,2,0)/255
    base = np.array(i["image"].convert("L"))
    base = np.stack([base,base,base]).transpose(1,2,0)/255
    f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
    ax1.imshow(combo)
    ax2.imshow(base)
    plt.show()